# Final Exam Problem 1

In [ ]:
import os
os.environ["PYTENSOR_FLAGS"] = "linker=py,cxx="

### 1. Fit a multinomial logistic regression to the data with the same prior from the lecture and provide the posterior summary output.

### 2. Find the probability that a customer is satisfied or very satisfied when the delay is 1 minute.

### 3. Give the odds ratio (and credible set) that a passenger would report ”Satisfied” vs. ”Very Dissatisfied” per minute of delay.

In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az

# 1. Load and clean data
df = pd.read_csv('bus.csv')

# Map the actual strings in bus.csv to 0–4
mapping = {
    'V.dis':   0,   # very dissatisfied
    'Dis':     1,   # dissatisfied
    'Neutral': 2,   # neutral
    'Satis':   3,   # satisfied
    'V.satis': 4    # very satisfied
}
df['y'] = df['satisfaction'].map(mapping)

# Drop any rows that failed to map
df = df.dropna(subset=['y'])
df['y'] = df['y'].astype(int)

# 2. Prepare predictor
X_raw = df['delay'].astype(float).values    # delays in {0,2,5,7}
Xc = (X_raw - X_raw.mean()) / X_raw.std()   # center & scale
X = Xc[:, None]                             # shape (n_samples, 1)

# 3. Build and sample the multinomial logistic model
with pm.Model() as model1:
    # Priors from lecture: Normal(0,10)
    α = pm.Normal('α', mu=0, sigma=10, shape=4)
    β = pm.Normal('β', mu=0, sigma=10, shape=4)

    # Logits: cat 0 has logit=0; cats 1–4 get α + β·X
    logits = pm.math.concatenate([
        pm.math.zeros((X.shape[0], 1)),
        α + β * X
    ], axis=1)

    # Likelihood
    obs = pm.Categorical(
        'obs',
        logit_p=logits,
        observed=df['y'].values
    )

    # Sampling
    trace1 = pm.sample(
        draws=4000,
        tune=4000,
        target_accept=0.99,
        init='adapt_diag',
        return_inferencedata=True
    )

# 4. Problem 1.1: Posterior summary for α and β
summary = az.summary(
    trace1,
    var_names=['α','β'],
    round_to=3
)[['mean','sd','hdi_3%','hdi_97%','ess_bulk','r_hat']]
print("1.1) Posterior summary for α and β:\n", summary)

# 5. Problem 1.2: P(satisfied or very satisfied | delay = 1)
α_samps = trace1.posterior['α'].stack(draws=('chain','draw')).values  # (4, N)
β_samps = trace1.posterior['β'].stack(draws=('chain','draw')).values  # (4, N)
n_draws = α_samps.shape[1]

def prob_each_category(delay_min):
    d0 = (delay_min - X_raw.mean()) / X_raw.std()
    z0 = np.zeros(n_draws)
    z_rest = α_samps + β_samps * d0
    z = np.vstack([z0, z_rest])                      # (5, n_draws)
    exp_z = np.exp(z - z.max(axis=0))
    return exp_z / exp_z.sum(axis=0)

p1 = prob_each_category(1)                           # (5, n_draws)
p_sat_or_vsat = p1[3] + p1[4]
mean_p = p_sat_or_vsat.mean()
hdi_low, hdi_high = az.hdi(p_sat_or_vsat, hdi_prob=0.95)
print(f"\n1.2) P(satisfied or very satisfied | delay=1) = {mean_p:.3f}")
print(f"     95% CI = [{hdi_low:.3f}, {hdi_high:.3f}]")

# 6. Problem 1.3: Odds‐ratio for “Satisfied” vs “Very Dissatisfied”
# slope for category 3 (“satisfied”) is β_samps[2]
log_or = β_samps[2]
or_mean = np.exp(log_or).mean()
or_low, or_high = np.exp(az.hdi(log_or, hdi_prob=0.95))
print(f"\n1.3) OR(satisfied vs very dissatisfied) per minute = {or_mean:.2f}")
print(f"     95% CI = [{or_low:.2f}, {or_high:.2f}]")


1.1 The posterior summaries for α and β coefficients show that all r̂ values are approximately 1.000 and effective sample sizes are large (>5000), indicating good convergence. The mean estimates are close to zero with relatively wide 95% highest density intervals (HDIs), reflecting moderate uncertainty in the delay effect on satisfaction categories.

1.2
The estimated probability that a customer is "Satisfied" or "Very Satisfied" when the delay is 1 minute is 0.401, with a 95% credible interval of [0.135, 0.678].

1.3 
The estimated odds ratio that a passenger reports "Satisfied" versus "Very Dissatisfied" per additional minute of delay is 1.39, with a 95% credible interval of [0.22, 5.04], suggesting a slight positive but uncertain association.

# Final Exam Problem 2

In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pytensor.tensor as pt
import numpy.ma as ma
import warnings
warnings.simplefilter("ignore")

# === Data Loading ===
df = pd.read_csv('pima.csv')
X = df.drop(columns=['test'])
y = df['test']

# Standardization
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Save mean-imputed version for Part 1 and SSVS
X_mean_imputed = X_scaled.fillna(X_scaled.mean())


In [ ]:
with pm.Model() as model_part1:
    intercept = pm.Normal('intercept', 0, 1)
    coefs = pm.Normal('coefs', 0, 1, shape=X_mean_imputed.shape[1])

    logits = intercept + pm.math.dot(X_mean_imputed.values, coefs)
    
    pm.Bernoulli('outcome', logit_p=logits, observed=y.values)
    
    trace_part1 = pm.sample(2000, tune=1000, target_accept=0.9, cores=1, random_seed=11)

summary_part1 = az.summary(trace_part1, hdi_prob=0.95)
print("Summary for 2.1 (Mean Imputation):\n", summary_part1)

significant_2_1 = summary_part1[(summary_part1['hdi_2.5%'] > 0) | (summary_part1['hdi_97.5%'] < 0)]
print("\nSignificant predictors (2.1):\n", significant_2_1)

In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pytensor.tensor as pt
import numpy.ma as ma
import warnings
warnings.simplefilter("ignore")

# === Data Loading ===
df = pd.read_csv('pima.csv')
X = df.drop(columns=['test'])
y = df['test']

# Standardization
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Save mean-imputed version for Part 1 and SSVS
X_mean_imputed = X_scaled.fillna(X_scaled.mean())

# === 2.2 - Logistic Regression with Bayesian Imputation (Cleaned) ===
predictors = X_scaled.columns

# Set up coordinates
coords_part2 = {"predictor": predictors, "obs_id": df.index}
n_predictors = X_scaled.shape[1]  # fixed here

with pm.Model(coords=coords_part2) as model_part2:
    # Priors for missing data imputation
    mu_imp = pm.Normal("mu_imp", mu=0, sigma=1, dims="predictor")
    sigma_imp = pm.HalfNormal("sigma_imp", sigma=1, dims="predictor")
    
    # Imputed predictors (treating missing values probabilistically)
    X_imputed = pm.Normal(
        "X_imputed",
        mu=mu_imp,
        sigma=sigma_imp,
        observed=X_scaled.values,  # fixed here
        dims=("obs_id", "predictor")
    )

    # Logistic regression part
    intercept = pm.Normal("intercept", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=2.5, dims="predictor")

    mu = intercept + pt.dot(X_imputed, beta)

    # Likelihood
    likelihood = pm.Bernoulli("likelihood", logit_p=mu, observed=y.values, dims="obs_id")
    
    # Faster and robust sampling
    trace_part2 = pm.sample(
        draws=10, 
        tune=500, 
        chains=4, 
        cores=4, 
        target_accept=0.95, 
        random_seed=22
    )

# === Summarize results for 2.2 ===
summary_part2 = az.summary(trace_part2, var_names=["intercept", "beta"], hdi_prob=0.95)
print("\nPosterior Summary (2.2 - Bayesian Imputation):")
print(summary_part2)

# Identify significant predictors
significant_vars_part2 = summary_part2[
    (summary_part2["hdi_2.5%"] > 0) | (summary_part2["hdi_97.5%"] < 0)
]
print("\nSignificant Variables (2.2 - HDI excludes 0):")
print(significant_vars_part2)
